# Create PDF

Imports

In [ ]:
# NEED TO CLEAN IMPORTS!

from reportlab.platypus import Table, SimpleDocTemplate, TableStyle, Paragraph, Image, PageBreak, Frame
from reportlab.lib import colors
from reportlab.lib import utils
from reportlab.lib.units import cm

from os import listdir

import csv

from reportlab.lib.pagesizes import landscape, A4
from reportlab.platypus.doctemplate import NextPageTemplate, PageTemplate, BaseDocTemplate

Definitions

In [ ]:
# makeTable takes data and creates a table with border and grid
def makeTable(data):
    table = Table(data)
    table.setStyle(TableStyle([('INNERGRID', (0,0), (-1,-1), 0.25, colors.black),('BOX', (0,0), (-1,-1), 0.75, colors.black)]))
    return table

In [ ]:
# https://stackoverflow.com/questions/5327670/image-aspect-ratio-using-reportlab-in-python
def get_image(path, width=1*cm):
    img = utils.ImageReader(path)
    iw, ih = img.getSize()
    aspect = ih / float(iw)
    return Image(path, width=width, height=(width * aspect))

# NOT USED
def get_rimage(path, width=1*cm):
    img = utils.ImageReader(path)
    iw, ih = img.getSize()
    aspect = ih / float(iw)
    return RotatedImage(path, width=width, height=(width * aspect))

In [ ]:
# Pagenumber:
# https://code.activestate.com/recipes/546511-page-x-of-y-with-reportlab/
# https://code.activestate.com/recipes/576832/

# Header:
# https://stackoverflow.com/questions/49344094/reportlab-pass-custom-argument-to-canvas#49350581
# (specifically Barranka's answer)

from reportlab.pdfgen import canvas
from reportlab.lib.units import mm

class NumberedCanvas(canvas.Canvas):
    def __init__(self, *args, **kwargs):
        canvas.Canvas.__init__(self, *args, **kwargs)
        self._saved_page_states = []
        self.header = ID_global

    def showPage(self):
        self._saved_page_states.append(dict(self.__dict__))
        self._startPage()

    def save(self):
        """add page info to each page (page x of y)"""
        num_pages = len(self._saved_page_states)
        for state in self._saved_page_states:
            self.__dict__.update(state)
            self.draw_page_number(num_pages)
            self.draw_header()
            canvas.Canvas.showPage(self)
        canvas.Canvas.save(self)

    def draw_page_number(self, page_count):
        self.setFont("Helvetica", 7)
        self.drawRightString(200*mm, 20*mm,
            "Page %d of %d" % (self._pageNumber, page_count))

    def draw_header(self):
        self.setFont("Helvetica", 12)
        self.drawString(100*mm, 280*mm, self.header)

# NOT USED: Can't get it to work, using templates instead!
# Rotate:
# https://stackoverflow.com/questions/29848988/a-simple-method-for-rotate-images-in-reportlab

# Center:
# https://stackoverflow.com/questions/43644835/reportlab-how-to-center-an-image-on-canvas
# (note, their solution used canvas)
class RotatedImage(Image):

    def wrap(self,availWidth,availHeight):
        h, w = Image.wrap(self,availHeight,availWidth)
        return w, h
    
    def draw(self):
        self.canv.rotate(90)
        Image.draw(self)

In [ ]:
# NOT USED
#https://stackoverflow.com/questions/14491169/reportlab-variable-nextpagetemplates

class fltpDocTemplate(BaseDocTemplate):
    def __init__(self, *args, **kwargs):
        BaseDocTemplate.__init__(self, *args, **kwargs)
    
    def afterPage(self):
        self._handle_nextPageTemplate('portrait')

In [ ]:
# NOT USED

def make_portrait(canvas,doc):
    canvas.setPageSize(A4)

def make_landscape(canvas,doc):
    canvas.setPageSize(landscape(A4))

General approach:
- Create document (doc)
- Create empty list (elements)
- Append to list
- Build doc

We can therefor make a forloop over the participants, fetch the relevant data and create a pdf for each participant. As of now, we use a handselected ID. We also have to move png and csv files manually, but this would be easy to change later.

### Current problem:
I'm able to start with a normal A4 page and then switch to landscape. One would think it would be easy to simply reverse the order (start with landscape and switch to portrait), but this leads to several issues I'm not able to pin down and correct. The first page still starts as a portrait, but the size is set to landscape, which means elements are missplaced. Changing to portrait then does nothing. Changing to landscape leads to improvements, but there is still some odd placement of elements.

In [ ]:
datafolder = '../data/'

png_width_full = 15
png_width_lap = 19

global ID_global
global fullplot_graph

for ID in listdir(datafolder):
    ID_global = ID
    output_location_string = datafolder+ID+'/'
    doc = BaseDocTemplate('report{}.pdf'.format(ID), pagesize=A4, 
                        rightMargin=25, leftMargin=25, topMargin=75, bottomMargin=25)
    elements = []

    # Used to change template in the middle of document
    # https://stackoverflow.com/questions/50660395/reportlab-how-to-change-page-orientation/50660701
    # https://stackoverflow.com/questions/5913682/reportlab-how-to-switch-between-portrait-and-landscape
    portrait_frame = Frame(doc.leftMargin, doc.bottomMargin, doc.width, doc.height, id='portrait_frame')
    landscape_frame = Frame(doc.leftMargin, doc.bottomMargin, doc.height, doc.width, id='landscape_frame')
    doc.addPageTemplates([PageTemplate(id='portrait', frames=portrait_frame, pagesize=A4),
                      PageTemplate(id='landscape', frames=landscape_frame, pagesize=landscape(A4)),
                      ])
    
    #Add full race graph
    elements.append(get_image(output_location_string+'fullracepd{}.png'.format(ID), width=png_width_full*cm))

    #Add table
    with open(output_location_string+'correlation_{}.csv'.format(ID)) as c:
        data = list(csv.reader(c))
        elements.append(makeTable(data))
    
    #Switch template (change happens on next page, hence before pagebreak)
    elements.append(NextPageTemplate('landscape'))
    elements.append(PageBreak())

    #Find all lap plots
    keyword = 'lap'
    lap_plot_list = []
    for lap_plot_file in listdir(output_location_string):
        if keyword in lap_plot_file:
            lap_plot_list.append(lap_plot_file)

    #Sort and add graphs
    lap_plot_list.sort()
    for lap_graph in lap_plot_list:
        elements.append(get_image(output_location_string+lap_graph, width=png_width_lap*cm))

    doc.build(elements, canvasmaker=NumberedCanvas)